In [25]:
!pip install --upgrade transformers==4.28.0 accelerate datasets zstandard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface

In [27]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [28]:
from datasets import load_dataset

ds = load_dataset("Multi-Domain-Expert-Layers/pubmed_abstracts", split="validation_domain")
ds = eli5.flatten()
ds[0]

{'text': 'Effect of sleep quality on memory, executive function, and language performance in patients with refractory focal epilepsy and controlled epilepsy versus healthy controls - A prospective study.\nWe aimed to evaluate the effect of sleep quality on memory, executive function, and language performance in patients with refractory focal epilepsy and controlled epilepsy and compare these with healthy individuals. We prospectively enrolled 37 adolescent and adult patients with refractory focal epilepsy (Group 1) and controlled epilepsy (Group 2) in each group. History pertaining to epilepsy and sleep were recorded, and all patients underwent overnight polysomnography. Language, memory, and executive function assessments were done using Western Aphasia Battery, Post Graduate Institute (PGI) memory scale, and battery of four executive function tests (Trail Making Test A & B, Digit symbol test, Stroop Task, and Verbal Fluency Test), respectively. Forty age- and sex-matched controls wer

In [29]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]], truncation=True, max_length=1024)

tokenized_ds = ds.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=ds.column_names,
)

Map (num_proc=4):   0%|          | 0/29860 [00:00<?, ? examples/s]

In [30]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [31]:
lm_dataset = tokenized_ds

training_args = TrainingArguments(
    output_dir="perplexity-results"
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=lm_dataset,
    data_collator=data_collator,
)

In [32]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")



You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Perplexity: 11.77
